# Profiling in Python

There are several different ways to measure resource usage in Python applications, which help identify where an application should be parallelized and what kinds of design patterns from Ray core to use.
to show how to determine where to parallelize.
We'll introduce profiling methods to measure memory allocation, objects creation, deterministic profiling (call stack), sampling, etc., plus different means of visualization.

The following example uses a Monte Carlo method for approximating the value of π, based on the tutorial ["Ray Crash Course - Tasks"](https://github.com/anyscale/academy/blob/main/ray-crash-course/01-Ray-Tasks.ipynb) by Dean Wampler.

We'll compare/contrast the serial implementation versus use of *remote functions* to help parallelize this application.
Then we'll analysze the per-CPU speedup, and also consider the overhead costs of using Ray core features -- to understand more about the trade-offs being made.

In [ ]:
%load_ext watermark
%watermark -v -m

## Launch Ray

First, start Ray and open its dashboard in another browser tab:

In [ ]:
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

## Application Code

Define a simple function that uses a stochastic method of approximating π, repeated through some number of samples:

In [ ]:
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statistics
import time

def estimate_pi (num_samples):
    xs = np.random.uniform(low=-1.0, high=1.0, size=num_samples)   # generate num_samples random samples for the x coordinate
    ys = np.random.uniform(low=-1.0, high=1.0, size=num_samples)   # generate num_samples random samples for the y coordinate
    xys = np.stack((xs, ys), axis=-1)                              # similar to Python's "zip(a,b)"; creates np.array([(x1,y1), (x2,y2), ...]).

    inside = (xs**2.0 + ys**2.0) <= 1.0                            # create a predicate over all the array elements
    xys_inside = xys[inside]                                       # select only those  elements inside the circle
    in_circle = xys_inside.shape[0]                                # return the number of elements inside the circle
    approx_pi = 4.0 * in_circle / num_samples                      # the Pi estimate

    return approx_pi

Since this approximation requires many samples, which are independent, a *task-parallel* pattern can be applied. We'll create a *remote function* version of the sampling function:

In [ ]:
@ray.remote
def distrib_estimate_pi (num_samples):
    return estimate_pi(num_samples)

Another function will collect measures for one epoch, i.e., some number of trials:

In [ ]:
def run_epoch (num_samples, num_trials, distrib=False):
    start = time.time()
    
    if distrib:
        refs = [distrib_estimate_pi.remote(num_samples) for _ in range(num_trials)]
        pis = ray.get(refs)
    else:
        pis = [estimate_pi(num_samples) for _ in range(num_trials)]

    # measure CPU time for the code section parallelized as a remote function
    duration = time.time() - start

    approx_pi = statistics.mean(pis)
    stdev = statistics.stdev(pis)
    error = 100.0 * abs(approx_pi - np.pi) / np.pi

    return num_samples, duration, approx_pi, stdev, error

Define class to manage the simulation:

In [ ]:
class Sim:
    DF_COL_NAMES = ["n", "duration", "approx_pi", "stdev", "error"]
    NUM_SAMPLES = 1000000
    NUM_TRIALS = 20
    STEP_SIZE = int(NUM_SAMPLES / 25)

    def __init__ (self, distrib=False, num_samples=NUM_SAMPLES, num_trials=NUM_TRIALS, step_size=STEP_SIZE):
        self.distrib = distrib
        self.num_samples = num_samples
        self.num_trials = num_trials
        self.step_size = step_size
        self.df = None


    def run (self):
        # use a minimum of 2 trials, to be able to calculate standard deviation
        results = [
            run_epoch(n_samp, self.num_trials, distrib=self.distrib)
            for n_samp in range(2, self.num_samples, self.step_size)
        ]
    
        self.df = pd.DataFrame(results, columns=self.DF_COL_NAMES)
        return self


    def plot (self):
        plt.plot("n", "duration", data=self.df, color="green", linewidth=1, linestyle="dashed")
        plt.plot("n", "error", data=self.df, color="red", linewidth=0.5, linestyle="dashed")
        plt.plot("n", "stdev", data=self.df, color="blue", linewidth=2)

        plt.yscale("log")
        plt.legend()
        plt.show()
        return self

## Profiling Tools

Next, we'll set up to use the following tools for profiling in Python:

  * [`objgraph`](https://mg.pov.lt/objgraph/)
  * [`tracemalloc`](https://docs.python.org/3/library/tracemalloc.html)
  * [`prun`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-prun)
  * [`snakeviz`](https://jiffyclub.github.io/snakeviz/)
  * [`pyinstrument`](https://github.com/joerick/pyinstrument/)

In [ ]:
import objgraph
import tracemalloc
import pyinstrument

%load_ext snakeviz

### Object Creation

The first method of profiling shown here uses
[`objgraph`](https://mg.pov.lt/objgraph/)
to examine a before/after contrast of what objects are getting generated by the application.
While other profiling methods can be repeated, this kind of analysis requires some special handling (isolation) and should be run first -- before other tools make the runtime environment confusing to analyze.
The objectives here are to understand:

  1. Which kinds of objects are growing (by count)
  2. How the different objects refer to each other

That can help identify if there are problems with allocating many objects, which might require the application code to be reworked.

First we'll measure a baseline of the top 10 kinds of objects that have been created so far:

In [ ]:
objgraph.show_growth(limit=10)

Then run the simulation serially **once**:

In [ ]:
%%time

sim_s = Sim(distrib=False, num_samples=Sim.NUM_SAMPLES)
sim_s.run();

Now we can measure the object counts again, and examine the growth (by contrast):

In [ ]:
objgraph.show_growth()

In [ ]:
objgraph.show_most_common_types() 

The top few categories are function calls, dictionaries, and tuples – which is expected, given how our π approximations create lots of NumPy arrays.
As expected, there's nothing much there to worry about – although in applications which large memory use we might need to parallelize so that each unit of work had more available memory.

Next, let's examine a graph of how object make reference to other objects.
This can be useful for tracing potential memory leaks...

In [ ]:
roots = objgraph.get_leaking_objects()
objgraph.show_refs(roots[:3], refcounts=True)

### Memory Allocation

The next profiling method uses
[`tracemalloc`](https://docs.python.org/3/library/tracemalloc.html)
to trace the memory blocks allocated by Python.
This analysis provides the following information:

  * traceback to where an object got allocated
  * statistics about allocated memory blocks per filename, per line: total size, number and average size
  * computing the differences between two snapshots to detect memory leaks
  
We'll run the application serially to capture a snapshot:

In [ ]:
tracemalloc.start()

Sim(distrib=False, num_samples=Sim.NUM_SAMPLES).run()

snapshot = tracemalloc.take_snapshot()

In [ ]:
top_stats = snapshot.statistics('traceback')

# pick the biggest memory block
for stat in top_stats:
    print("%s memory blocks: %.1f KiB" % (stat.count, stat.size / 1024))

    for line in stat.traceback.format():
        print(line)

There should not be any *memory leaks* in this application.

### Deterministic Profiling

The
[`%%prun` magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-prun)
invokes the
[cProfile](https://docs.python.org/3/library/profile.html)
*deterministic* profiler in Python to trace how often and for what duration the different functions get called.
In other words, we'll track of the call stack statistics to understand more about CPU use.
This configures the profiler to save data to the file `prof_cpu.txt` as text.

The [`%%snakeviz` magic](https://jiffyclub.github.io/snakeviz/)
then analyzes that profiling data and generates an interactive report.

Note that *deterministic profiling* creates some overhead which can distort the measurements in relatively small code blocks or short-running code, although it's generally find for long-running programs.

In [ ]:
%%prun -q -T prof_cpu.txt
%%snakeviz

Sim(distrib=False, num_samples=Sim.NUM_SAMPLES).run()

The *icicle* interactive chart illustrates how so very much of the overall CPU cost is in the `estimate_pi()` method.

### Sampling Profiler

The
[`pyinstrument`](https://github.com/joerick/pyinstrument/)
library provides a *sampling profiler* – an alternative way to measure the call stack and CPU use.
While not *exact*, this approach creates less overhead:

In [ ]:
profiler = pyinstrument.Profiler()
profiler.start()

Sim(distrib=False, num_samples=Sim.NUM_SAMPLES).run()

profiler.stop()
display(HTML(profiler.output_html()))

Again, the `estimate_pi()` function is where most of the CPU cost occurs, although we can get clearer measures for how much overhead there is among the rest of code.
In this case the cumulative overhead is relatively small, probably measured in milliseconds.

## Results of Serial Execution

Now let's look at a visualization of the *number of samples* plotted versus *duration*, *error*, and *standard deviation*:

In [ ]:
sim_s.plot();

Note how the *standard deviation* drops quickly.
We can calculate that measure without even knowing an "exact" value of π (the *error* measure), so we could potentially rework this application to have an *early termination* by setting a threshold on the *stdev* measure.

In [ ]:
sim_s.df.describe()

## Results of Parallel Execution

Now let's run again and this time parallelize the application using *remote functions*, by setting the `distrib=True` flag.
We'll use the `%%time` magic to show the "wall clock" CPU time:

In [ ]:
%%time

sim_d = Sim(distrib=True, num_samples=Sim.NUM_SAMPLES)
sim_d.run();

In [ ]:
sim_d.plot();

In [ ]:
sim_d.df.describe()

While the *duration* of `estimate_pi()` CPU use scales pretty much the same as in serial execution, we'll show in the next section how the parallel run improves the application speed dramatically – depending on the number of available CPUs.

## Visualizing Aggregate Measures

Now let's build a pipeline in `scikit-learn` to run a 
[*polynomial regression*](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)
of the *duration* measures.
In other words, we'll perform some "curve fitting" to calculate the speedup of using *remote functions*.

The `degree` parameter is set to the value `1`, which means we'll look at a linear regression.
This pipeline allows you to evaluate fitting to higher order polynomials, by changing this parameter.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

DEGREE = 1
polyreg = make_pipeline(PolynomialFeatures(DEGREE), LinearRegression())

X = sim_s.df.iloc[:, 0].values.reshape(-1, 1)
Y_s = sim_s.df.iloc[:, 1].values.reshape(-1, 1)
Y_d = sim_d.df.iloc[:, 1].values.reshape(-1, 1)

polyreg.fit(X, Y_s)
Y_s_pred = polyreg.predict(X)

polyreg.fit(X, Y_d)
Y_d_pred = polyreg.predict(X)

In [ ]:
plt.scatter(X, Y_s, color="lightgray")
plt.scatter(X, Y_d, color="gray")

plt.plot(X, Y_s_pred, color="green", linewidth=1, label="serial")
plt.plot(X, Y_d_pred, color="red", linewidth=0.5, label="distrib")

plt.legend()
plt.show()

The distributed processing shows less time required for the tasks in aggregate.
Let's calculate how much speedup, based on a ratio of the *slope* values for the two fitted regressions:

In [ ]:
m = (Y_s_pred / Y_d_pred)
m

Now we can plot these ratios, and take their median value:

In [ ]:
plt.plot(m, color="red", linewidth=1, label="speedup {:.2f}".format(np.median(m)))
plt.legend()
plt.show()

Depending on the number of available CPUs, you should see an asymptotically linear increase in performance.

In [ ]:
ray.shutdown()

## Exercise

Try running the `Sim` simulation again in its parallelized mode, with a much larger number of samples.
While that's running, check the Ray dashboard to see how processing for the *remote tasks* gets distributed across the available CPUs.